### `Step-6` : Image Augmentation
- After completing some preprocessing steps in PySpark on Ubuntu, I downloaded the image dataset—consisting of around 1,000 images across 8 classes—from the HDFS system to my local disk. I then transferred the dataset from Ubuntu to Windows for the next phase. In this step, I planned to apply data augmentation techniques to simulate real-life challenges for the model. However, before augmentation, I decided to perform the annotation (labeling) process first. In object detection tasks, proper annotation significantly contributes to the model’s performance and learning. While annotation can also be done after augmentation, doing so would have resulted in around 2,000 images to label, which would be very time-consuming. Therefore, I chose to manually annotate the initial 1,000 images before applying augmentation.
- For annotation (labeling), I used Label Studio, a powerful and user-friendly tool that supports various annotation formats, including the YOLO format. It was particularly helpful for creating accurate bounding boxes for object detection tasks. The following commands were used to install and launch Label Studio via the Anaconda Prompt:
  - pip install label-studio
  - label-studio start
    
- After manually labeling all images, I downloaded the dataset in YOLO format to prepare for the training phase. The downloaded folder contains four main components: Images, Labels, Classes and JSON file ( not required for training). Each image has a corresponding label file with the same name. The YOLO label format includes:

`class_id`	Rrepresenting the class (e.g., 0 for bear, 1 for crocodile, etc.)

`x_center`	Horizontal center of the bounding box (relative to image width)

`y_center`	Vertical center of the bounding box (relative to image height)

`width`	    Width of the bounding box (relative to image width)

`height`	Height of the bounding box (relative to image height)

All values are normalized (scaled between 0 and 1) with respect to the image size.

 

- There is only one step left before model training which is augmentation. I applied four different augmentation method :
  - `180 degree rotation :` I created 200 randomly selected images from the raw dataset and applied a 180-degree rotation to each. One of the challenging aspects of this step was updating the corresponding label files, as the bounding box coordinates change after rotation. For 180-degree rotation, I was able to successfully generate new label coordinates by adjusting the YOLO format values accordingly.I initially attempted 90 and 270-degree rotations as well, but I faced difficulties in recalculating the correct label coordinates for those angles. As a result, I decided to proceed only with 180-degree rotated images to ensure annotation accuracy.
  - `Blurring :` I applied strong Gaussian blur to 200 randomly selected images from the dataset. The goal of this augmentation step was to train the model to detect objects even when they appear with low pixel clarity, simulating real-world scenarios such poor camera focus. Since the structure and position of objects in the images remain unchanged, this process did not require recalculating or modifying the label coordinates.
  - `Brightness and Contrast :` I applied strong brightness and contrast adjustments to 200 randomly selected images from the dataset. The goal of this augmentation step was to train the model to effectively detect objects under varying lighting conditions, such as overly bright or dim environments. Since these adjustments do not alter the position or shape of the objects, the original label coordinates remained valid and did not require any modification.
  - `Motion Blur` I applied a strong motion blur kernel to 200 randomly selected images from the dataset. The aim of this augmentation was to train the model to detect objects accurately in scenarios where either the camera or the object is in motion. This helps improve the model’s robustness in real-time applications, especially when dealing with moving subjects. Since motion blur does not change the object’s position or size, the original label coordinates remained valid.

### Augmentation for 180 degree rotation 

In [ ]:
import os
import cv2
import random
import numpy as np
from glob import glob

# Paths
INPUT_FOLDER = r"C:\Users\mesut\Desktop\yolo"  # Original dataset folder
OUTPUT_FOLDER = r"C:\Users\mesut\Desktop\yolo_rotation"  # Augmented dataset folder
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Rotation function with YOLO label correction
def rotate_image_and_labels(image, labels, angle):
    if angle == 180:
        image = cv2.rotate(image, cv2.ROTATE_180)
        new_img_height, new_img_width = image.shape[:2]

        new_labels = []
        for cls_id, x_center, y_center, w, h in labels:
            new_x = 1 - x_center
            new_y = 1 - y_center
            new_w = w
            new_h = h

            # Convert to pixel values
            x_pixel = new_x * new_img_width
            y_pixel = new_y * new_img_height
            w_pixel = new_w * new_img_width
            h_pixel = new_h * new_img_height

            # Convert back to normalized values
            new_x = x_pixel / new_img_width
            new_y = y_pixel / new_img_height
            new_w = w_pixel / new_img_width
            new_h = h_pixel / new_img_height

            # Keep within valid range
            new_x = max(0, min(1, new_x))
            new_y = max(0, min(1, new_y))
            new_w = max(0, min(1, new_w))
            new_h = max(0, min(1, new_h))

            new_labels.append([cls_id, new_x, new_y, new_w, new_h])

        return image, new_labels

    return image, labels

# Process Images and Labels
image_paths = glob(os.path.join(INPUT_FOLDER, '**/*.jpg'), recursive=True)
selected_images = random.sample(image_paths, 200)  # Select 200 random images

for img_path in selected_images:
    img = cv2.imread(img_path)

    # Load corresponding YOLO label file
    label_path = img_path.replace('.jpg', '.txt')
    if not os.path.exists(label_path):
        continue

    with open(label_path, 'r') as file:
        labels = []
        for line in file.readlines():
            values = line.strip().split()
            if len(values) == 5:
                labels.append(list(map(float, values)))

    if not labels:
        continue

    # Rotate by 180 degrees only
    rotated_img, rotated_labels = rotate_image_and_labels(img, labels, 180)

    # Save Augmented Image and Labels with modified names
    relative_path = os.path.relpath(img_path, INPUT_FOLDER)
    base_name = os.path.splitext(os.path.basename(img_path))[0]
    aug_img_path = os.path.join(OUTPUT_FOLDER, os.path.dirname(relative_path), f"{base_name}_rotated(180).jpg")
    aug_label_path = aug_img_path.replace('.jpg', '.txt')

    os.makedirs(os.path.dirname(aug_img_path), exist_ok=True)
    cv2.imwrite(aug_img_path, rotated_img)

    with open(aug_label_path, 'w') as file:
        for label in rotated_labels:
            file.write(f"{int(label[0])} {label[1]:.15f} {label[2]:.15f} {label[3]:.15f} {label[4]:.15f}\n")

print("✅ 180-degree rotation complete! 200 images successfully rotated and saved.")


### Augmentation for Blurring

In [ ]:
# Paths
INPUT_FOLDER = r"C:\Users\mesut\Desktop\yolo"  # Original dataset folder
OUTPUT_FOLDER = r"C:\Users\mesut\Desktop\yolo_blur"  # Augmented dataset folder
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Blurring function
def apply_blur(image):
    # Apply very strong Gaussian blur with even larger kernel sizes (21, 25, 31)
    kernel_size = random.choice([21, 25, 31])
    blurred_image = cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)
    return blurred_image

# Process Images and Labels
image_paths = glob(os.path.join(INPUT_FOLDER, '**/*.jpg'), recursive=True)
selected_images = random.sample(image_paths, 200)  # Select 200 random images

for img_path in selected_images:
    img = cv2.imread(img_path)

    # Load corresponding YOLO label file
    label_path = img_path.replace('.jpg', '.txt')
    if not os.path.exists(label_path):
        continue

    with open(label_path, 'r') as file:
        labels = []
        for line in file.readlines():
            values = line.strip().split()
            if len(values) == 5:
                labels.append(list(map(float, values)))

    if not labels:
        continue

    # ✅ Apply extra-strong blurring augmentation
    blurred_img = apply_blur(img)

    # Save Augmented Image and Labels with modified names
    relative_path = os.path.relpath(img_path, INPUT_FOLDER)
    base_name = os.path.splitext(os.path.basename(img_path))[0]
    aug_img_path = os.path.join(OUTPUT_FOLDER, os.path.dirname(relative_path), f"{base_name}_blurred.jpg")
    aug_label_path = aug_img_path.replace('.jpg', '.txt')

    os.makedirs(os.path.dirname(aug_img_path), exist_ok=True)
    cv2.imwrite(aug_img_path, blurred_img)

    with open(aug_label_path, 'w') as file:
        for label in labels:
            file.write(f"{int(label[0])} {label[1]:.15f} {label[2]:.15f} {label[3]:.15f} {label[4]:.15f}\n")

print("✅ Extra-strong blurring augmentation complete! 200 images successfully blurred and saved.")


### Augmentation for Brightness and Contrast

In [ ]:
# Paths
INPUT_FOLDER = r"C:\Users\mesut\Desktop\yolo"  # Original dataset folder
OUTPUT_FOLDER = r"C:\Users\mesut\Desktop\yolo_bright_and_contr"  # Augmented dataset folder
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Brightness and Contrast adjustment function
def adjust_brightness_contrast(image):
    # Stronger brightness and contrast adjustment
    alpha = random.uniform(0.2, 3.0)  # Contrast control (0.2–3.0)
    beta = random.randint(-100, 100)   # Brightness control (-100 to 100)
    adjusted = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    return adjusted

# Process Images and Labels
image_paths = glob(os.path.join(INPUT_FOLDER, '**/*.jpg'), recursive=True)
selected_images = random.sample(image_paths, 200)  # Select 200 random images

for img_path in selected_images:
    img = cv2.imread(img_path)

    # Load corresponding YOLO label file
    label_path = img_path.replace('.jpg', '.txt')
    if not os.path.exists(label_path):
        continue

    with open(label_path, 'r') as file:
        labels = []
        for line in file.readlines():
            values = line.strip().split()
            if len(values) == 5:
                labels.append(list(map(float, values)))

    if not labels:
        continue

    # ✅ Apply stronger brightness and contrast adjustment
    adjusted_img = adjust_brightness_contrast(img)

    # Save Augmented Image and Labels with modified names
    relative_path = os.path.relpath(img_path, INPUT_FOLDER)
    base_name = os.path.splitext(os.path.basename(img_path))[0]
    aug_img_path = os.path.join(OUTPUT_FOLDER, os.path.dirname(relative_path), f"{base_name}_bright_contrast.jpg")
    aug_label_path = aug_img_path.replace('.jpg', '.txt')

    os.makedirs(os.path.dirname(aug_img_path), exist_ok=True)
    cv2.imwrite(aug_img_path, adjusted_img)

    with open(aug_label_path, 'w') as file:
        for label in labels:
            file.write(f"{int(label[0])} {label[1]:.15f} {label[2]:.15f} {label[3]:.15f} {label[4]:.15f}\n")

print("✅ Strong brightness and contrast augmentation complete! 200 images successfully adjusted and saved.")

### Augmentation for Motion Blur

In [ ]:
# Paths
INPUT_FOLDER = r"C:\Users\mesut\Desktop\yolo"  # Original dataset folder
OUTPUT_FOLDER = r"C:\Users\mesut\Desktop\yolo_motion_blur"  # Augmented dataset folder
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Moderate Motion Blur function
def apply_motion_blur(image):
    # Moderate motion blur with smaller kernel sizes
    kernel_size = random.choice([15, 20, 25])  # Reduced kernel size for softer blur
    angle = random.choice([0, 45, 90, 135])  # Horizontal, vertical, and diagonal motion blur

    # Create motion blur kernel
    kernel = np.zeros((kernel_size, kernel_size))
    if angle == 0:  # Horizontal motion blur
        kernel[int((kernel_size - 1) / 2), :] = np.ones(kernel_size)
    elif angle == 90:  # Vertical motion blur
        kernel[:, int((kernel_size - 1) / 2)] = np.ones(kernel_size)
    elif angle == 45:  # Diagonal motion blur (top-left to bottom-right)
        np.fill_diagonal(kernel, 1)
    elif angle == 135:  # Diagonal motion blur (top-right to bottom-left)
        np.fill_diagonal(np.fliplr(kernel), 1)

    kernel = kernel / kernel_size

    # Apply motion blur
    motion_blurred = cv2.filter2D(image, -1, kernel)
    return motion_blurred

# Process Images and Labels
image_paths = glob(os.path.join(INPUT_FOLDER, '**/*.jpg'), recursive=True)
selected_images = random.sample(image_paths, 200)  # Select 200 random images

for img_path in selected_images:
    img = cv2.imread(img_path)

    # Load corresponding YOLO label file
    label_path = img_path.replace('.jpg', '.txt')
    if not os.path.exists(label_path):
        continue

    with open(label_path, 'r') as file:
        labels = []
        for line in file.readlines():
            values = line.strip().split()
            if len(values) == 5:
                labels.append(list(map(float, values)))

    if not labels:
        continue

    # ✅ Apply moderate motion blur augmentation
    motion_blurred_img = apply_motion_blur(img)

    # Save Augmented Image and Labels with modified names
    relative_path = os.path.relpath(img_path, INPUT_FOLDER)
    base_name = os.path.splitext(os.path.basename(img_path))[0]
    aug_img_path = os.path.join(OUTPUT_FOLDER, os.path.dirname(relative_path), f"{base_name}_motion_blur.jpg")
    aug_label_path = aug_img_path.replace('.jpg', '.txt')

    os.makedirs(os.path.dirname(aug_img_path), exist_ok=True)
    cv2.imwrite(aug_img_path, motion_blurred_img)

    with open(aug_label_path, 'w') as file:
        for label in labels:
            file.write(f"{int(label[0])} {label[1]:.15f} {label[2]:.15f} {label[3]:.15f} {label[4]:.15f}\n")

print("✅ Moderate motion blur augmentation complete! 200 images successfully blurred and saved.")
